In [73]:
from minio import Minio
import pandas as pd
import os
import io
from datetime import datetime, timedelta


In [68]:
client = Minio(endpoint="localhost:9000", access_key='a5926TSNVC2r9J4Y2Eqh', secret_key='3YBQqcerjz5TsV8X851gi3Rl7YNclYQ6UD1MrEPY', secure=False)

In [70]:
#DEFINITIONS
tbl_list = ['users', 'products', 'transactions', 'transaction_detail']
bucket_name = 'snapshot'

#DEFINE CURRENT TIME VARIABLES
current_dt_string = datetime.now()
current_date_full = current_dt_string.strftime("%d%m%Y%H%M%S")
current_year = current_dt_string.strftime("%Y")
current_month = current_dt_string.strftime("%m")
current_date = current_dt_string.strftime("%d")

#DEFINE PREVIOUS DAY TIME VARIABLES

prev_dt_string = current_dt_string - timedelta(days=1)
prev_full_date = prev_dt_string.strftime("%d%m%Y%H%M%S")
prev_year = prev_dt_string.strftime("%Y") # prev_year means that the year that last day's data was on, not literally 'last year'
prev_month = prev_dt_string.strftime("%m") # the same rule applies for prev_month and prev_date.
prev_date = prev_dt_string.strftime("%d")


In [93]:
read_the_parquet= pd.read_parquet("/home/admin/Downloads/products_snapshot_08062024111632.parquet")

In [94]:
read_the_parquet

,id,product_name,unit_price,category


In [89]:

def create_df_for_t1_day():
    df_t1 = []
    for table in tbl_list:
        t1_dt_obj_list = client.list_objects(bucket_name=bucket_name, prefix=f'{bucket_name}/{table}/2024/06/08/')
        # t1_dt_obj_list = client.list_objects(bucket_name=bucket_name, prefix=f'{bucket_name}/{table}/{prev_year}/{prev_month}/{prev_date}/')
        for obj in t1_dt_obj_list:            
            file_path = os.path.basename(obj.object_name)
            object_key = obj.object_name
            print(f'object_key: {object_key}, file_path: {file_path}')

            try:
                response_t1 = client.get_object(bucket_name,object_name=object_key)
                buffer = io.BytesIO(response_t1.read())
                df = pd.read_parquet(buffer)
                df_t1.append(df)
                print(df_t1)       
            
            finally:
                response_t1.close()
                buffer.close()
                response_t1.release_conn()
    return df_t1

In [91]:
df_t1 = create_df_for_t1_day()

object_key: snapshot/users/2024/06/08/users_snapshot_08062024111632.parquet, file_path: users_snapshot_08062024111632.parquet
[Empty DataFrame
Columns: [id, name, email, created_at]
Index: []]
object_key: snapshot/products/2024/06/08/products_snapshot_08062024111632.parquet, file_path: products_snapshot_08062024111632.parquet
[Empty DataFrame
Columns: [id, name, email, created_at]
Index: [], Empty DataFrame
Columns: [id, product_name, unit_price, category]
Index: []]
object_key: snapshot/transactions/2024/06/08/transactions_snapshot_08062024111632.parquet, file_path: transactions_snapshot_08062024111632.parquet
[Empty DataFrame
Columns: [id, name, email, created_at]
Index: [], Empty DataFrame
Columns: [id, product_name, unit_price, category]
Index: [], Empty DataFrame
Columns: [id, transaction_date, total_amount, cash_received, change_due, user_id]
Index: []]
object_key: snapshot/transaction_detail/2024/06/08/transaction_detail_snapshot_08062024111632.parquet, file_path: transaction_de

In [92]:
df_t1[1]

,id,product_name,unit_price,category


In [74]:
def create_df_for_t2_day():
    for table in tbl_list:
        t2_dt_obj_list = client.list_objects(bucket_name=bucket_name, prefix=f'{bucket_name}/{table}/{prev_year}/{prev_month}/{prev_date}/')
        for obj in t2_dt_obj_list:            
            file_path = os.path.basename(obj.object_name)
            object_key = obj.object_name
            print(f'object_key: {object_key}, file_path: {file_path}')

            try:
                response_t2 = client.get_object("snapshot", object_key=object_key)
                buffer = io.BytesIO(response_t2.read())
                df_t2 = pd.read_parquet(data=buffer)
                print(df_t2)       
                
            finally:
                response_t2.close()
                buffer.close()
                response_t2.release_conn()
    return df_t2

In [77]:
create_df_for_t1_day()
create_df_for_t2_day()

UnboundLocalError: cannot access local variable 'df_t1' where it is not associated with a value